<a href="https://colab.research.google.com/github/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial Prompting and Fine-Tuning Pre-trained Generative Language Models**
*Johny Moreira, Altigran da Silva, Luciano Barbosa*

SIMPÓSIO BRASILEIRO DE BANCO DE DADOS - 2023


# Configurações básicas do ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install openai
! pip install langchain
! pip install python-dotenv

# https://gpt4all.io/index.html
# "Um ecossistema open-source para treino e deployment de LLMs"
! pip install gpt4all

In [ ]:
def print_smaller_lines(text, length):
    for i in range(0, len(text), length):
        print(text[i:i+length])

# Carregando os pacotes necessários

In [ ]:
import os
import openai
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

import dotenv
dotenv.load_dotenv('/content/drive/MyDrive/Tutorial SBBD/Colab/.env')

True

In [ ]:
project_path = "/content/drive/MyDrive/Tutorial SBBD/Colab/"
dados_dir = project_path + "Dados/"

In [ ]:
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORGANIZATION")

# Prompting em um Modelo de Linguagem carregado localmente com o LangChain

In [ ]:
template = """Question: {question}\nAnswer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
local_path = (
    "/content/drive/MyDrive/Tutorial SBBD/Colab/models/llama-2-7b-chat.ggmlv3.q4_0.bin"
)

In [ ]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
llm = GPT4All(model=local_path, backend="llama", callbacks=callbacks, verbose=True)

Found model file at  /content/drive/MyDrive/Tutorial SBBD/Colab/models/llama-2-7b-chat.ggmlv3.q4_0.bin


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "Beyonce was a member of which famous girl band?"

llm_chain.run(question)

 Beyoncé was not in any girl bands. She started her career as the lead singer of Destiny's Child, but she has since become a successful solo artist.

" Beyoncé was not in any girl bands. She started her career as the lead singer of Destiny's Child, but she has since become a successful solo artist."

In [ ]:
template = """Question: {question}
Answer: Think step-by-step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "Beyonce was a member of which famous girl band?"
llm_chain.run(question)

 Beyoncé was not in any girl group, she started her career as the lead singer of Destiny's Child before pursuing a successful solo career.

" Beyoncé was not in any girl group, she started her career as the lead singer of Destiny's Child before pursuing a successful solo career."

# Engenharia de Prompts com LangChain e ChatGPT

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=256)

## 1 - Injetando Padrões

### 1.1 Extração de Informação

In [ ]:
input_prompt = PromptTemplate(
    input_variables=["texto"],
    template =
    'TEXTO: {texto}\n' \
    'Dado o texto acima, extraia informações importantes no formato abaixo:\n' \
    '<CHAVE>: <VALOR>'\
    'Preserve a exata formatação apresentada.'
)

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

chain = LLMChain(llm=llm, prompt=input_prompt)
result = chain.run(texto=texto)
print_smaller_lines(result, 120)


Nome: Alan Mathison Turing
Data de nascimento: 23 de junho de 1912
Local de nascimento: Londres
Data de falecimento: 7 d
e junho de 1954
Local de falecimento: Wilmslow, Cheshire
Profissões: Matemático, cientista da computação, lógico, cripto
analista, filósofo e biólogo teórico
Contribuições: Desenvolvimento da moderna ciência da computação teórica, formalizaç
ão dos conceitos de algoritmo e computação com a máquina de Turing, considerado o pai da ciência da computação teórica e
 da inteligência artificial
Fatores que afetaram seu reconhecimento: Homossexualidade e cobertura de seu trabalho pela L
ei de Segredos Oficiais.


### 1.2 Análise de Sentimentos

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

In [ ]:
input_prompt1 = PromptTemplate(
    input_variables=["texto"],
    template =
    'Classifique os exemplos a seguir de acordo com as ' \
    'seguintes polaridades Positivo, Negativo e Neutro.\n' \
    'EXEMPLOS:\n{texto}'
)

In [ ]:
chain = LLMChain(llm=llm, prompt=input_prompt1)
result = chain.run(texto=texto)
print(result)

1 - Positivo
2 - Neutro
3 - Negativo
4 - Negativo
5 - Positivo


In [ ]:
input_prompt2 = PromptTemplate(
    input_variables=["texto"],
    template =
    'Classifique os exemplos a seguir de acordo com as polaridades Positivo, Negativo e Neutro.'\
    'Utilize o seguinte formato ###DECLARAÇÃO: <DECLARAÇÃO>\n###POLARIDADE: <POLARIDADE>.\n' \
    '{texto}'
)
chain = LLMChain(llm=llm, prompt=input_prompt2)
result = chain.run(texto=texto)
print(result)

###DECLARAÇÃO: Minha Experiência na loja foi incrível.
###POLARIDADE: Positivo.

###DECLARAÇÃO: Eu acho que podiam melhorar o produto.
###POLARIDADE: Neutro.

###DECLARAÇÃO: O atendimento foi horrível!
###POLARIDADE: Negativo.

###DECLARAÇÃO: Não volto mais.
###POLARIDADE: Negativo.

###DECLARAÇÃO: Recomendo demais a banoffe. É uma delícia!
###POLARIDADE: Positivo.


### 1.3 Perguntas e Respostas

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=512)

In [ ]:
input_prompt = PromptTemplate(
    input_variables=["texto"],
    template =
    'Dado o texto a seguir: {texto}\n' \
    'Gere quatro questões e suas respectivas respostas utilizando o template abaixo.\n'\
    'Preserve a exata formatação do template apresentado:'\
    'PERGUNTA: <pergunta gerada>'\
    'RESPOSTA: <resposta>'
)

In [ ]:
texto = 'Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954) '\
        'foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico '\
        'britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação '\
        'teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina '\
        'de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente '\
        'considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas '\
        'realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser '\
        'homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.'

chain = LLMChain(llm=llm, prompt=input_prompt)
result = chain.run(texto=texto)
print_smaller_lines(result, 100)

PERGUNTA: Quais foram as áreas de atuação de Alan Turing?
RESPOSTA: Alan Turing atuou como matemátic
o, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico.

PERGUNTA: Qual foi 
a contribuição de Alan Turing para a ciência da computação?
RESPOSTA: Alan Turing foi altamente infl
uente no desenvolvimento da moderna ciência da computação teórica, proporcionando uma formalização d
os conceitos de algoritmo e computação com a máquina de Turing.

PERGUNTA: Por que Alan Turing é con
siderado o pai da ciência da computação teórica e da inteligência artificial?
RESPOSTA: Alan Turing 
é considerado o pai da ciência da computação teórica e da inteligência artificial devido às suas con
tribuições significativas no desenvolvimento dessas áreas, como a formalização dos conceitos de algo
ritmo e computação com a máquina de Turing.

PERGUNTA: Por que Alan Turing não foi totalmente reconh
ecido em seu país de origem durante sua vida?
RESPOSTA: Alan Turing não foi totalmente reco

## 2 - Padrão Persona


*   Introduz ao modelo uma representação fictícia ou não, normalmente a de um especialista em um domínio ou personalidade específica

*   É útil quando não se tem certeza sobre os detalhes exatos da saída esperada mas você sabe quem procurar para lhe entregar o resultado esperado



In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=512)

### 2.1 Um advogado especialista

In [ ]:
texto = 'Escreva como se fosse um advogado brasileiro especialista em direito penal.'\
        'Pontue de forma resumida as possíveis penas para um caso de lesão corporal leve sem contexto de violência doméstica.'

result = llm.predict(texto)
print_smaller_lines(result, 120)

Como advogado especialista em direito penal, é importante ressaltar que as penas para um caso de lesão corporal leve sem
 contexto de violência doméstica podem variar de acordo com as circunstâncias específicas do caso e a legislação vigente
 no Brasil. No entanto, de forma geral, as possíveis penas podem incluir:

1. Pena de detenção: A pena de detenção pode 
variar de 3 meses a 1 ano, dependendo da gravidade da lesão e das circunstâncias do caso.

2. Prestação de serviços à co
munidade: Além da pena de detenção, o juiz pode determinar a prestação de serviços à comunidade como forma de punição ao
 agressor.

3. Multa: O agressor também pode ser condenado ao pagamento de multa, cujo valor será estabelecido pelo juiz
, considerando a capacidade financeira do agressor e a gravidade da lesão.

4. Suspensão condicional do processo: Em alg
uns casos, o juiz pode optar por suspender condicionalmente o processo, desde que o agressor cumpra determinadas condiçõ
es, como não se envolver em novo

### 2.2 Um crítico gastronômico

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=512)

texto = 'Você é um crítico gastronômico e está muito irritado com o tipo de serviço e atendimento de um restaurante que acabou de visitar.'\
        'Escreva uma curta review do estabelecimento.'

result = llm.predict(texto)
print_smaller_lines(result, 120)

Review do Restaurante "Sabor Amargo"

Minha experiência no restaurante "Sabor Amargo" foi extremamente decepcionante. De
sde o momento em que entrei, fui recebido com uma falta de cortesia e profissionalismo por parte da equipe de atendiment
o. O garçom parecia mais interessado em conversar com os colegas do que em nos atender.

Além disso, o serviço foi extre
mamente lento e desorganizado. Tive que esperar por mais de 30 minutos para receber o cardápio, e quando finalmente fize
mos o pedido, os pratos demoraram uma eternidade para chegar à mesa. Durante toda a espera, nenhum membro da equipe se p
reocupou em nos informar sobre o atraso ou oferecer qualquer tipo de compensação.

Quanto à comida, fiquei igualmente de
sapontado. Os pratos chegaram frios e sem sabor, como se tivessem sido preparados com pressa e sem cuidado. A apresentaç
ão também deixou muito a desejar, com os pratos parecendo desleixados e pouco apetitosos.

Além disso, o ambiente do res
taurante era extremamente barulh

### 2.3 Um Astrofísico

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=512)

texto = 'Escreva como se fosse um astrofísico. Me explique por que o universo está expandindo.'

result = llm.predict(texto)
print_smaller_lines(result, 120)

A expansão do universo é um fenômeno fascinante e complexo que tem sido objeto de estudo e debate entre os astrofísicos 
ao longo das últimas décadas. Para entender por que o universo está se expandindo, é necessário compreender alguns conce
itos fundamentais da cosmologia.

De acordo com a teoria do Big Bang, o universo teve origem em um estado extremamente d
enso e quente, há aproximadamente 13,8 bilhões de anos. Desde então, o universo tem passado por um processo contínuo de 
expansão. Essa expansão é evidenciada pela observação de que as galáxias estão se afastando umas das outras em todas as 
direções.

A principal explicação para a expansão do universo é a presença de uma forma de energia misteriosa chamada en
ergia escura. A energia escura é uma força repulsiva que age como uma espécie de antigravidade, empurrando as galáxias p
ara longe umas das outras. Ela é responsável por cerca de 70% da composição total do universo.

A energia escura é uma d
as maiores incógnitas da astrofí

In [ ]:
texto = 'Você é um astrofísico. Me explique por que o universo está expandindo. Me explique como se eu tivesse 12 anos.'

result = llm.predict(texto)
print_smaller_lines(result, 120)

Claro! Vou explicar de uma forma simples e fácil de entender.

Imagine que o universo é como uma pizza, mas em vez de te
r uma borda fixa, ela está sempre crescendo. As fatias de pizza representam as galáxias, que são grupos de estrelas, pla
netas e outros objetos cósmicos.

Agora, pense que você está no centro da pizza, em uma fatia de queijo. Se você olhar p
ara as outras fatias, vai perceber que todas estão se afastando de você. Isso acontece porque o espaço entre as fatias e
stá se expandindo.

Essa expansão do universo foi descoberta por astrônomos observando as galáxias. Eles perceberam que,
 em geral, as galáxias estão se afastando umas das outras. Isso significa que o espaço entre elas está ficando maior.

M
as por que isso está acontecendo? Bem, os cientistas ainda estão estudando isso, mas acredita-se que seja resultado de u
ma grande explosão chamada Big Bang, que aconteceu há cerca de 13,8 bilhões de anos.

No momento do Big Bang, todo o uni
verso estava concentrado em um p

## 3 - Refinamento de Perguntas

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=512)

conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

def chatbot(texto):
    res = conversation.predict(input=texto)
    return res

texto = 'Sempre que eu fizer uma pergunta relacionada a jurisprudência brasileira, '\
          'sugira uma pergunta mais refinada considerando as especificidades do '\
          'direito brasileiro. Pergunte se eu gostaria de utilizar a pergunta sugerida.'
while True:
  response = conversation.predict(input=texto)
  print("CHATGPT: ", response)

  texto = input("USER: ")
  if texto.lower() == 'sair':
    break

CHATGPT:  Claro, posso fazer isso! Sempre que você fizer uma pergunta relacionada à jurisprudência brasileira, posso sugerir uma pergunta mais refinada levando em consideração as especificidades do direito brasileiro. Em seguida, perguntarei se você gostaria de utilizar a pergunta sugerida. Isso parece bom para você?
USER: posso ser preso por lesão corporal?
CHATGPT:  De acordo com o Código Penal Brasileiro, a lesão corporal é considerada um crime. No entanto, a resposta para a sua pergunta depende de vários fatores, como a gravidade da lesão, as circunstâncias em que ocorreu e se houve intenção de causar dano. Além disso, é importante considerar a jurisprudência brasileira, que pode fornecer orientações mais específicas sobre casos semelhantes. Posso sugerir uma pergunta mais refinada considerando as especificidades do direito brasileiro?
USER: sim
CHATGPT:  Com base nas informações fornecidas, uma pergunta mais refinada considerando as especificidades do direito brasileiro poderia se

CHATGPT:  Claro, posso fazer isso! Sempre que você fizer uma pergunta relacionada à jurisprudência brasileira, posso sugerir uma pergunta mais refinada levando em consideração as especificidades do direito brasileiro. Em seguida, perguntarei se você gostaria de utilizar a pergunta sugerida. Isso parece bom para você?

USER: posso ser preso por lesão corporal?

CHATGPT:  De acordo com o Código Penal Brasileiro, a lesão corporal é considerada um crime. No entanto, a resposta para a sua pergunta depende de vários fatores, como a gravidade da lesão, as circunstâncias em que ocorreu e se houve intenção de causar dano. Além disso, é importante considerar a jurisprudência brasileira, que pode fornecer orientações mais específicas sobre casos semelhantes. Posso sugerir uma pergunta mais refinada considerando as especificidades do direito brasileiro?

USER: sim

CHATGPT:  Com base nas informações fornecidas, uma pergunta mais refinada considerando as especificidades do direito brasileiro poderia ser: "Quais são os critérios utilizados pela jurisprudência brasileira para determinar se uma lesão corporal é considerada grave o suficiente para resultar em prisão?". Gostaria de utilizar essa pergunta sugerida?

USER: sim

CHATGPT:  Ótimo! Vou registrar sua pergunta sugerida: "Quais são os critérios utilizados pela jurisprudência brasileira para determinar se uma lesão corporal é considerada grave o suficiente para resultar em prisão?". Agora, vou pesquisar a resposta para você. Por favor, aguarde um momento.

USER: aguardando...

CHATGPT:  Após realizar uma pesquisa, encontrei algumas informações relevantes sobre os critérios utilizados pela jurisprudência brasileira para determinar se uma lesão corporal é considerada grave o suficiente para resultar em prisão. De acordo com a jurisprudência, a gravidade da lesão é avaliada levando em consideração fatores como a extensão do dano físico, a duração da incapacidade para o trabalho, a necessidade de tratamento médico e a possibilidade de sequelas permanentes. Além disso, a jurisprudência também considera as circunstâncias em que a lesão ocorreu, como a presença de violência doméstica, a relação entre agressor e vítima, e se houve intenção de causar dano. No entanto, é importante ressaltar que cada caso é analisado individualmente pelos tribunais, levando em consideração todas as circunstâncias específicas.

USER: sair

## 4 - Padrão de Verificação Cognitiva
Divide perguntas complexas em subperguntas menores e gerenciáveis.

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=256)

texto = 'Em um caso de agressão corporal o indivíduo agredido sofreu sequelas '\
        'permanentes e encontra-se impossibilitado de trabalhar. O agressor poderá ser sentenciado ' \
        'à prisão e ao pagamento de indenização vitalícia? Considere a legislação brasileira.'

result = llm.predict(texto)
print_smaller_lines(result, 120)

Sim, de acordo com a legislação brasileira, o agressor poderá ser sentenciado à prisão e ao pagamento de indenização vit
alícia no caso de agressão corporal que resulte em sequelas permanentes e impossibilidade de trabalhar para o indivíduo 
agredido.

A agressão corporal é considerada um crime no Brasil, previsto no Código Penal. Dependendo da gravidade da ag
ressão, o agressor pode ser enquadrado em diferentes tipos penais, como lesão corporal leve, grave, gravíssima ou seguid
a de morte.

Além da pena de prisão, o agressor também pode ser condenado a pagar uma indenização à vítima. Essa indeniz
ação tem o objetivo de compensar os danos físicos, psicológicos e materiais causados pela agressão. No caso de sequelas 
permanentes que impossibilitem o trabalho, a indenização pode ser vitalícia, ou seja, paga mensalmente ao longo da vida 
da vítima.

É importante ressaltar que cada caso é analisado individualmente pelo juiz, levando em consideração


In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=1024)

texto = 'Separe a pergunta abaixo em subperguntas e suas respectivas respostas. '\
        'Ao final, combine as respostas das subperguntas e as sumarize em uma única resposta final refinada.'\
        'Pergunta: Em um caso de agressão corporal o indivíduo agredido sofreu sequelas '\
        'permanentes e encontra-se impossibilitado de trabalhar. O agressor poderá ser sentenciado ' \
        'à prisão e ao pagamento de indenização vitalícia? Considere a legislação brasileira.'

result = llm.predict(texto)
print_smaller_lines(result, 120)

Subperguntas:

1. Em um caso de agressão corporal, o agressor pode ser sentenciado à prisão?
2. Em um caso de agressão c
orporal, o agressor pode ser obrigado a pagar uma indenização?
3. Em um caso de agressão corporal, a indenização pode se
r vitalícia?
4. Qual é a legislação brasileira que trata de casos de agressão corporal?

Respostas:

1. Sim, de acordo c
om a legislação brasileira, o agressor pode ser sentenciado à prisão em casos de agressão corporal, dependendo da gravid
ade do crime.
2. Sim, o agressor pode ser obrigado a pagar uma indenização à vítima em casos de agressão corporal, de ac
ordo com a legislação brasileira.
3. Sim, em casos de agressão corporal que resultem em sequelas permanentes e impossibi
lidade de trabalhar, a indenização pode ser vitalícia, conforme a legislação brasileira.
4. A legislação brasileira que 
trata de casos de agressão corporal é o Código Penal Brasileiro, mais especificamente os artigos que tratam de lesão cor
poral e suas consequências.

Res

## 5 - Chain-of-Thought

Uma cadeia de prompts interconectados pode estimular o raciocínio nos modelos de linguagem.

FONTE: https://arxiv.org/pdf/2201.11903.pdf

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=1024)

texto = 'Q: Roger tem 5 bolas de tênis. Ele compra mais 2 pacotes de bolas de tênis.
'Cada pacote tem 2 bolas de tênis. Quantas bolas de tênis Roger tem agora?'\
        'A: Roger tinha 5 bolas de tênis. 2 pacotes com 3 bolas de tênis em cada
    'um dá um total de 6 bolas de tênis. 5 + 6 = 11. A resposta é 11.'\
        '\nQ: A cafeteria tinha 23 maçãs. Se eles usaram 20 delas para fazer uma
        'torta e depois compraram mais 6 maçãs, quantas maçãs tem na cafeteria?'

result = llm.predict(texto)
print_smaller_lines(result, 120)

A: A cafeteria tinha inicialmente 23 maçãs. Se eles usaram 20 delas para fazer uma torta, restaram 23 - 20 = 3 maçãs. De
pois, eles compraram mais 6 maçãs, então agora a cafeteria tem 3 + 6 = 9 maçãs. A resposta é 9.


In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1, max_tokens=1024)

texto = 'Q: Os números ímpares no grupo a seguir quando somados resultam em um
'número par: 4, 8, 9, 15, 12, 2, 1.'\
        '\nA: Somar todos os números ímpares (9, 15, 1) resulta em 25.
        '25 é um número ímpar. Portanto a assertiva anterior é Falsa.'\

        '\nQ: Os números ímpares no grupo a seguir quando somados resultam
        'em um número par: 15, 32, 5, 13, 82, 7, 1.'

result = llm.predict(texto)
print_smaller_lines(result, 120)

A: Somar todos os números ímpares (15, 5, 13, 7, 1) resulta em 41. 41 é um número ímpar. Portanto a assertiva anterior é
 Falsa.


# Demo do Falcon
[DEMO](https://huggingface.co/spaces/tiiuae/falcon-180b-demo)